In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
matplotlib.rcParams['svg.fonttype'] = 'none' # for svgs
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from matplotlib import cm

import flexiznam as flz
from cottage_analysis.analysis import spheres, common_utils
from cottage_analysis.plotting import basic_vis_plots
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_map.figure_utils import depth_selectivity, closed_loop_rsof, get_session_list
from v1_depth_map.figure_utils import common_utils as v1_common_utils

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
READ_VERSION = 10
VERSION = 10
READ_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{READ_VERSION}"
SAVE_ROOT = flz.get_data_root("processed", flexilims_session=flexilims_session) / "v1_manuscript_figures"/f"ver{VERSION}"
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"supp").mkdir(parents=True, exist_ok=True)
reload=False

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
session_name_openloop = "PZAH6.4b_S20220524"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example_openloop, trials_df_example_openloop = spheres.sync_all_recordings(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example_openloop = pipeline_utils.create_neurons_ds(
    session_name=session_name_openloop,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example_openloop = pd.read_pickle(neurons_ds_example_openloop.path_full)

In [ ]:
mouse_list = flz.get_entities("mouse", flexilims_session=flexilims_session)
mouse_list = mouse_list[
    mouse_list.name.isin(
        [
            "PZAH6.4b",
            "PZAG3.4f",
            "PZAH8.2h",
            "PZAH8.2i",
            "PZAH8.2f",
            "PZAH10.2d",
            "PZAH10.2f",
        ]
    )
]

session_list = get_session_list.get_sessions(
    flexilims_session,
    exclude_openloop=False,
    exclude_pure_closedloop=False,
    mouse_list=mouse_list,
)

# recalculate with precompute_data/calculate_rs_stats_all_sessions.py
if reload:
    results_all = depth_selectivity.get_rs_stats_all_sessions(
        flexilims_session,
        session_list,
        nbins=60,
        rs_thr_min=None,
        rs_thr_max=None,
        still_only=False,
        still_time=1,
        corridor_length=6,
        blank_length=3,
        overwrite=False,
    )
    results_all.to_pickle(SAVE_ROOT/"supp"/"results_all_rs_supp.pickle")
else:
    results_all = pd.read_pickle(READ_ROOT/"supp"/"results_all_rs_supp.pickle")

In [ ]:
fig = plt.figure(figsize=(18 / 2.54, 21 / 2.54))
# 3 example neurons closed loop vs open loop matrices
EXAMPLE_ROIS_OPENLOOP = [121,357,440]
fontsize_dict = {"title": 5, "label": 7, "tick": 5, "legend": 5}
ylims=[[-0.01,0.7],[-0.01,0.6],[-0.01,0.5]]
for iroi, roi in enumerate(EXAMPLE_ROIS_OPENLOOP):
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) - 1):
        xlabel = ""
    else:
        xlabel = "Running speed (cm/s)"
    if iroi != (len(EXAMPLE_ROIS_OPENLOOP) // 2):
        ylabel = ""
    else:
        ylabel = "Optic flow speed (degrees/s)"
    # fig.add_axes([0.05, 0.75 - 0.3 * iroi, 0.17, 0.2])
    fig.add_axes([0.05, 0.7 - 0.15*iroi, 0.1, 0.1])
    depth_tuning_kwargs = dict(
        rs_thr=None,
        plot_fit=True,
        plot_smooth=False,
        linewidth=1.5,
        linecolor="royalblue",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
        markersize=8,
        markeredgecolor='w',
    )
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example_openloop,
        trials_df=trials_df_example_openloop,
        use_col="depth_tuning_popt_closedloop",
        roi=roi,
        ylim=ylims[iroi],
        **depth_tuning_kwargs,
)   
    if iroi != len(EXAMPLE_ROIS_OPENLOOP) - 1:
        plt.xlabel("")

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="",
        ylabel=ylabel,
        plot_x=0.2,
        plot_y=0.7 - 0.15 * iroi,
        plot_width=0.1,
        plot_height=0.1,
        cbar_width=None,
        fontsize_dict=fontsize_dict,
    )

    closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example_openloop,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=0,
        vmin=vmin,
        vmax=vmax,
        xlabel=xlabel,
        ylabel="",
        plot_x=0.35,
        plot_y=0.7 - 0.15 * iroi,
        plot_width=0.1,
        plot_height=0.1,
        cbar_width=0.005,
        fontsize_dict=fontsize_dict,
    )

# open loop actual vs virtual RS coorelation for 5 depths and 8 depths
results_all_5depths = results_all[(results_all["session"].iloc[:,0].str.contains("PZAH6.4b"))|(results_all["session"].iloc[:,0].str.contains("PZAG3.4f"))]
results_all_8depths = results_all[~((results_all["session"].iloc[:,0].str.contains("PZAH6.4b"))|(results_all["session"].iloc[:,0].str.contains("PZAG3.4f")))]
for iplot, (results, depth_list) in enumerate(zip(
    [results_all_5depths, results_all_8depths],
    [np.geomspace(0.06,6,5), np.geomspace(0.05,6.4,8)]
)):
    closed_loop_rsof.plot_openloop_rs_correlation_alldepths(results, 
                                                            depth_list=depth_list, 
                                                            fontsize_dict=fontsize_dict, 
                                        ax1=fig.add_axes([0.58, 0.62-0.23*iplot, 0.05, 0.18]), ax2=fig.add_axes([0.65, 0.62-0.23*iplot, 0.3, 0.18]),
                                        linewidth=2, elinewidth=2, jitter=0.2, scatter_markersize=2, scatter_alpha=0.4, capsize=5, capthick=2,
                                        ylim = (-0.5, 1))
    
plt.savefig(SAVE_ROOT / "fig_supp_openloop.svg", bbox_inches="tight", dpi=300)